In [1]:
!pip install -Uq "trl[peft]"  bitsandbytes transformers datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 15.6 MB/s eta 0:00:00


In [ ]:

from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import AutoModelForCausalLM,AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen2.5-1.5B-Instruct"


quantization_config_8bit = BitsAndBytesConfig(load_in_8bit=True,
                                              bnb_8bit_compute_dtype=torch.float16
                                              )
# quantization_config_4bit = BitsAndBytesConfig(load_in_4bit=True)


# model_4bit = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     quantization_config=quantization_config_4bit
# )

model_8bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config_8bit,
    trust_remote_code=True

)

# model 32 bits
# model_32bit = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto"
# )

tokenizer = AutoTokenizer.from_pretrained(model_name,padding_side="left",trust_remote_code=True)


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
import torch

def print_model_size_in_mb_gb(model):
    """
    Prints the total size of a PyTorch model (including parameters and buffers)
    in both MB and GB.
    """
    param_size = 0
    for param in model.parameters():
        param_size += param.numel() * param.element_size()

    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.numel() * buffer.element_size()

    size_bytes = param_size + buffer_size
    size_mb = size_bytes / (1024 ** 2)
    size_gb = size_mb / 1024

    print(f"Model total size: {size_mb:.2f} MB ({size_gb:.2f} GB)")


In [5]:
# print_model_size_in_mb_gb(model_4bit)
# print_model_size_in_mb_gb(model_32bit)
print_model_size_in_mb_gb(model_8bit)

Model total size: 1694.90 MB (1.66 GB)


In [6]:
# model_32bit

In [7]:
model_8bit

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear8bitLt(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Q

In [8]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
)

In [9]:
print(text)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>



load dataset

In [10]:
# now we have to load the docker dataset for finetunign of the large language model
from datasets import load_dataset
# Load dataset from the hub


dataset =  load_dataset("MattCoddity/dockerNLcommands",split="train")

README.md: 0.00B [00:00, ?B/s]

06102023.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2415 [00:00<?, ? examples/s]

In [11]:
dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 2415
})

In [12]:
from datasets import DatasetDict
train_test_Dataset = dataset.train_test_split(test_size=0.2,seed=42)
train_test_Dataset
dataset = DatasetDict(
    {
        "train": train_test_Dataset["train"],
        "validatation": train_test_Dataset["test"]
})

In [13]:
dataset


DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 1932
    })
    validatation: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 483
    })
})

In [14]:
print(dataset["train"][0]["input"])
print(dataset["train"][0]["output"])
print(dataset["train"][0]["instruction"])
# print(datasets["train"][""][1])

Find the repository, tag, and ID of the images that were created before the latest nginx image.
docker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"
translate this sentence in docker command


In [15]:
dataset["train"][0]

{'input': 'Find the repository, tag, and ID of the images that were created before the latest nginx image.',
 'output': 'docker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"',
 'instruction': 'translate this sentence in docker command'}

In [16]:
# convert to chat templet for the training of the mdoel
def convert_to_chatml(example):
    return {
        "messages": [
            {"role": "system", "content": example["instruction"]},
            {"role": "user", "content": example["input"]},
            {"role": "assistant", "content": example["output"]},
        ]
    }

In [17]:
print(convert_to_chatml(dataset["train"][0]))

{'messages': [{'role': 'system', 'content': 'translate this sentence in docker command'}, {'role': 'user', 'content': 'Find the repository, tag, and ID of the images that were created before the latest nginx image.'}, {'role': 'assistant', 'content': 'docker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"'}]}


In [18]:
dataset = dataset.map(convert_to_chatml)


Map:   0%|          | 0/1932 [00:00<?, ? examples/s]

Map:   0%|          | 0/483 [00:00<?, ? examples/s]

In [19]:
dataset["train"][0]

{'input': 'Find the repository, tag, and ID of the images that were created before the latest nginx image.',
 'output': 'docker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"',
 'instruction': 'translate this sentence in docker command',
 'messages': [{'content': 'translate this sentence in docker command',
   'role': 'system'},
  {'content': 'Find the repository, tag, and ID of the images that were created before the latest nginx image.',
   'role': 'user'},
  {'content': 'docker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"',
   'role': 'assistant'}]}

In [20]:
# tokenizer qwen
tokenizer.apply_chat_template(dataset["train"][0]["messages"],tokenize=False)

'<|im_start|>system\ntranslate this sentence in docker command<|im_end|>\n<|im_start|>user\nFind the repository, tag, and ID of the images that were created before the latest nginx image.<|im_end|>\n<|im_start|>assistant\ndocker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"<|im_end|>\n'

In [21]:
def convert_chat_temp(example):
  return {"messages":tokenizer.apply_chat_template(example["messages"],tokenize=False)}

In [22]:
dataset =dataset.map(convert_chat_temp)

Map:   0%|          | 0/1932 [00:00<?, ? examples/s]

Map:   0%|          | 0/483 [00:00<?, ? examples/s]

In [23]:
def text_to_token(example):
  return tokenizer(example["messages"])

In [24]:
dataset = dataset.map(text_to_token,batched=True,remove_columns=dataset["train"].column_names)
# now we have to delete the rest of the column which are not include in the train of the model

Map:   0%|          | 0/1932 [00:00<?, ? examples/s]

Map:   0%|          | 0/483 [00:00<?, ? examples/s]

In [25]:
len(dataset["train"][6]["input_ids"])

55

In [26]:
# What is a Data Collator?

# A Data Collator is a component in the data loading pipeline that prepares mini-batches of examples just before feeding them to the model.
# Think of it as a "batch organizer":
# It takes a list of tokenized samples (from your dataset),
# Pads them to the same length (for batching),
# Creates attention masks, labels, and sometimes special tokens,
# Returns the ready-to-train batch in a dictionary format that the model expects.

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="pt")
tokenizer.pad_token

'<|endoftext|>'

In [27]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset["train"], batch_size=32, shuffle=True,collate_fn=data_collator)
for step, batch in enumerate(train_dataloader):
    if step >= 3:
        break
    print(batch["input_ids"].shape)

torch.Size([32, 76])
torch.Size([32, 74])
torch.Size([32, 74])


In [28]:
next(iter(train_dataloader))["input_ids"].shape

torch.Size([32, 79])

### PEFT LORA
PEFT methods can be combined with TRL for fine-tuning to reduce memory requirements. We can pass the LoraConfig to the model when loading it.

In [29]:
from peft import LoraConfig

# TODO: Configure LoRA parameters
# r: rank dimension for LoRA update matrices (smaller = more compression)
rank_dimension = 32
# lora_alpha: scaling factor for LoRA layers (higher = stronger adaptation)
lora_alpha = 32
# lora_dropout: dropout probability for LoRA layers (helps prevent overfitting)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Rank dimension - typically between 4-32
    lora_alpha=lora_alpha,  # LoRA scaling factor - typically 2x rank
    lora_dropout=lora_dropout,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "gate_proj", "down_proj","up_proj"],  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
)

In [30]:
model_8bit

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear8bitLt(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Q

In [31]:
# now we have to load the lora
from peft import get_peft_model, LoraConfig, TaskType
import copy

model_8bit_clone = copy.deepcopy(model_8bit)
model_8bit_lora = get_peft_model(model_8bit_clone, peft_config)


In [32]:
model_8bit_lora.print_trainable_parameters()

trainable params: 34,177,024 || all params: 1,577,891,328 || trainable%: 2.1660


In [33]:
model_8bit

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear8bitLt(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Q

In [34]:
from trl import SFTConfig, SFTTrainer

In [35]:
output_dir = "Qwen-Instruct_Fine_For_Docker_Commands"
training_args = SFTConfig(
    #num_train_epochs=1,
    max_steps=500,                                         # Number of dataset passes. For full trainings, use `num_train_epochs` instead
    per_device_train_batch_size=2,                        # Batch size per GPU/CPU
    per_device_eval_batch_size=2,                         # Batch size for evaluation
    gradient_accumulation_steps=8,                        # Gradients are accumulated over multiple steps → effective batch size = 2 * 8 = 32
    eval_steps=20,
    learning_rate=3e-5,
    logging_steps=20,
    save_steps=100,
    eval_strategy="steps",
    warmup_steps=5,                                       # Gradually increase LR during first N steps
    optim="adamw_torch",                                   # Optimizer
    max_length=None,                                      # For VLMs, truncating may remove image tokens, leading to errors during training. max_length=None avoids it
    # Logging / reporting
    output_dir=output_dir,                                # Where to save model checkpoints and logs
    report_to='tensorboard',                                  # Experiment tracking tool
    # Hub integration
    # push_to_hub=True,
)

In [36]:

# Create SFTTrainer with LoRA configuration
trainer = SFTTrainer(
    model=model_8bit,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validatation"],
    data_collator=data_collator,
    peft_config=peft_config,  # LoRA configuration
    # max_seq_length=max_seq_length,  # Maximum sequence length
    processing_class=tokenizer,
)
# Start training
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 d

Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
20,3.157900,2.047884,1.959194,15850.000000,0.695985
40,1.500700,0.914310,0.968622,32002.000000,0.787541
60,0.846900,0.742208,0.747853,48377.000000,0.816053
80,0.710000,0.666217,0.679339,64434.000000,0.829243
100,0.647300,0.614250,0.650666,80487.000000,0.840004
120,0.631900,0.575418,0.624119,96730.000000,0.846971
140,0.569700,0.544019,0.548868,112603.000000,0.851789
160,0.527000,0.522145,0.526985,128911.000000,0.854186
180,0.511400,0.499053,0.492609,144704.000000,0.861447
200,0.490000,0.481270,0.511340,160968.000000,0.862173


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to 

TrainOutput(global_step=500, training_loss=0.6235162725448609, metrics={'train_runtime': 5740.5148, 'train_samples_per_second': 1.394, 'train_steps_per_second': 0.087, 'total_flos': 3648639010240512.0, 'train_loss': 0.6235162725448609, 'epoch': 4.132505175983437})

In [37]:

trainer.save_model("lora_adapters_model")
trainer.push_to_hub(dataset_name="dockerNLcommands",)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...r_Commands/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

  ...adapter_model.safetensors:   0%|          |  562kB /  137MB            

  ...195425.830d0585cea8.222.0:  22%|##1       | 6.02kB / 27.5kB            

  ...ommands/training_args.bin:  22%|##1       | 1.36kB / 6.22kB            

CommitInfo(commit_url='https://huggingface.co/azherali/Qwen-Instruct_Fine_For_Docker_Commands/commit/4c4897191def39e009a4c1972b55b7f9bc2b4657', commit_message='End of training', commit_description='', oid='4c4897191def39e009a4c1972b55b7f9bc2b4657', pr_url=None, repo_url=RepoUrl('https://huggingface.co/azherali/Qwen-Instruct_Fine_For_Docker_Commands', endpoint='https://huggingface.co', repo_type='model', repo_id='azherali/Qwen-Instruct_Fine_For_Docker_Commands'), pr_revision=None, pr_num=None)

In [38]:
model_8bit.save_pretrained('adapters_weights')

In [39]:
layer0 =  model_8bit.model.layers[0]
layer0.self_attn.q_proj.weight

# model weights are in 8 bit beacuase we have quantize the model to 8bit

Parameter containing:
Parameter(Int8Params([[ 39, -27,   4,  ...,  28,   7,  33],
            [  1,  -3, -23,  ..., -22,   6, -10],
            [ 16,  64, -15,  ..., -33,  31,  39],
            ...,
            [-14, -39,   0,  ...,   2, -15, -78],
            [-18,  40, -31,  ...,  65, -12, -27],
            [ 26, -59,  19,  ...,  55,  21,  47]], device='cuda:0',
           dtype=torch.int8))

In [40]:
layer0 =  model_8bit.model.layers[0]
weightA =layer0.self_attn.q_proj.lora_A["default"].weight
weightA
del weightA

In [41]:
import torch
from transformers import AutoModelForCausalLM
from peft import PeftModel

model_name = "Qwen/Qwen2.5-1.5B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    # quantization_config=quantization_config_8bit,
    trust_remote_code=True

)


base_model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotar

In [45]:
from peft import PeftModel
base_model = PeftModel.from_pretrained(base_model, "lora_adapters_model")

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.mlp.gate_proj.lor

In [46]:
base_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): Qwen2ForCausalLM(
          (model): Qwen2Model(
            (embed_tokens): Embedding(151936, 1536)
            (layers): ModuleList(
              (0-27): 28 x Qwen2DecoderLayer(
                (self_attn): Qwen2Attention(
                  (q_proj): lora.Linear(
                    (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1536, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=1536, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    

In [47]:
# 3. Merge adapter weights with base model
base_model = base_model.merge_and_unload()

In [48]:
base_model.push_to_hub("Qwen-Instruct_Finetune_For_Docker_Commands")
tokenizer.push_to_hub("Qwen-Instruct_Finetune_For_Docker_Commands")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors: 100%|##########|  40.0B /  40.0B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpgyd5exn8/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

CommitInfo(commit_url='https://huggingface.co/azherali/Qwen-Instruct_Finetune_For_Docker_Commands/commit/603248368b36f3981ab0edd66fc84f6fae96e10d', commit_message='Upload tokenizer', commit_description='', oid='603248368b36f3981ab0edd66fc84f6fae96e10d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/azherali/Qwen-Instruct_Finetune_For_Docker_Commands', endpoint='https://huggingface.co', repo_type='model', repo_id='azherali/Qwen-Instruct_Finetune_For_Docker_Commands'), pr_revision=None, pr_num=None)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "azherali/Qwen-Instruct_Finetune_For_Docker_Commands"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


adapter_config.json:   0%|          | 0.00/940 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [3]:


# List all containers with Ubuntu as their ancestor.
# docker ps --filter 'ancestor=ubuntu'

prompt = "List all containers with Ubuntu as their ancestor."
messages = [
    {"role": "system", "content": "translate this sentence in docker command"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

To list all containers that have Ubuntu as an ancestor, you can use the following Docker CLI command:

```sh
docker ps -a --filter "ancestor=ubuntu"
```

This command filters the `docker ps` output to show only those containers where the image is `ubuntu`. The `-a` flag ensures that it includes both running and stopped containers.
